# Satellite Solver

# Generate Plots Noise-Free Simulator

In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

size = 14
legendsize = 12

In [ ]:
df = pd.read_csv('../res_satellite_solver.csv', sep=',')
df

In [ ]:
ax1 = df.plot(x='num_qubits', y='calculation_time_qaoa', color='orange', label="QAOA")   
ax1.tick_params(which='both', labelsize=size)
ax2 = df.plot( x='num_qubits', y='calculation_time_wqaoa', color='blue', ax=ax1, label="W-QAOA") 
ax3 = df.plot( x='num_qubits', y='calculation_time_vqe', color='green', ax=ax1, label="VQE") 

plt.xlabel("Number of Locations", size=size)
plt.ylabel("Calculation Time", size=size)
plt.legend(fontsize=legendsize)
plt.savefig('sat_solver_time.pdf')
plt.show()

ax1 = df.plot(x='num_qubits', y='success_rate_qaoa', color='orange', label="QAOA")   
ax1.tick_params(which='both', labelsize=size)
ax2 = df.plot( x='num_qubits', y='success_rate_wqaoa', color='blue', ax=ax1, label="W-QAOA") 
ax3 = df.plot( x='num_qubits', y='success_rate_vqe', color='green', ax=ax1, label="VQE") 

plt.xlabel("Number of Locations", size=size)
plt.ylabel("Result Quality", size=size)
plt.legend(fontsize=legendsize)
plt.savefig('sat_solver_quality.pdf')
plt.show()

# Create Plots Noisy Simulator

In [ ]:
from qiskit import IBMQ
IBMQ.load_account()
provider = IBMQ.get_provider(hub="ibm-q", group="open", project="main")

## Pre-Process

In [ ]:
from mqt.problemsolver.satellitesolver import utils
def get_result_for_job_id(row):
    ac_reqs = utils.init_random_acquisition_requests(row.num_qubits)
    mdl = utils.create_satellite_doxplex(ac_reqs)
    converter, qubo = utils.convert_docplex_to_qubo(mdl)

    qaoa_result = provider.runtime.job(row.job_id_qaoa).result()
    qaoa_most_likely_eigenstate = utils.sample_most_likely(qaoa_result.get("eigenstate"))

    qaoa_validity = utils.check_solution(ac_reqs=ac_reqs, solution_vector=qaoa_most_likely_eigenstate)
    if qaoa_validity:
        qaoa_val = utils.get_success_ratio(ac_reqs, qubo, qaoa_most_likely_eigenstate)
    else:
        qaoa_val =  -1
    vqe_result = provider.runtime.job(row.job_id_vqe).result()
    vqe_most_likely_eigenstate = utils.sample_most_likely(vqe_result.get("eigenstate"))

    vqe_validity = utils.check_solution(ac_reqs=ac_reqs, solution_vector=vqe_most_likely_eigenstate)
    if vqe_validity:
        vqe_val = utils.get_success_ratio(ac_reqs, qubo, vqe_most_likely_eigenstate)
    else:
        vqe_val =  -1
    return (qaoa_val, vqe_val)

In [ ]:
df = pd.read_csv('../res_satellite_solver_noisy.csv', sep=',')
df

In [ ]:
res = df.apply(get_result_for_job_id, axis=1)
df['success_rate_qaoa_device'] = res[0][0]
df['success_rate_vqe_device'] = res[0][1]
df

## Graphs

In [ ]:
ax1 = df.plot(x='num_qubits', y='success_rate_qaoa', color='orange', linestyle="dashed", label="QAOA: Sim")   
ax3 = df.plot( x='num_qubits', y='success_rate_qaoa_device', color='orange', linestyle="dotted", ax=ax1, label="QAOA: ibmq_nairobi") 
ax2 = df.plot( x='num_qubits', y='success_rate_vqe', color='green', ax=ax1, linestyle="dashed", label="VQE: Sim") 
ax4 = df.plot( x='num_qubits', y='success_rate_vqe_device', color='green', ax=ax1, linestyle="dotted", label="VQE: ibmq_nairobi") 
ax1.tick_params(which='both', labelsize=size)

plt.xlabel("Number of Locations", size=size)
plt.ylabel("Result Quality", size=size)
plt.legend(fontsize=legendsize)
plt.savefig('sat_solver_quality_noisy.pdf')
plt.show()

In [ ]:
from qiskit import IBMQ
from qiskit.algorithms.optimizers import SPSA
from qiskit.tools import job_monitor

from mqt.problemsolver.satellitesolver.evaluator import eval_vqe_using_qiskit_runtime

IBMQ.load_account()

In [ ]:
provider = IBMQ.get_provider(hub="ibm-q", group="open", project="main")

In [ ]:
provider.backends()

## Imports

In [ ]:
from mqt import ddsim
import numpy as np

from mqt.problemsolver.satellitesolver import utils
from mqt.problemsolver.satellitesolver.utils import  QAOA, W_QAOA

from qiskit import Aer, execute
from qiskit_optimization.converters.quadratic_program_to_qubo import QuadraticProgramToQubo
from qiskit_optimization.translators import from_docplex_mp
from qiskit_optimization.converters.quadratic_program_to_qubo import (
    QuadraticProgramToQubo,
)
from qiskit.algorithms.optimizers import L_BFGS_B, ADAM, SPSA

import matplotlib.pyplot as plt

import warnings
import time

## Create arbitrary Satellite Problem Instance

In [ ]:
num_requests = 7
ac_reqs = utils.init_random_acquisition_requests(num_requests)
utils.plot_acqisition_requests(ac_reqs)

## Create Docplex Model and Convert it to QUBO Formulation

In [ ]:
penalty = None
mdl = utils.create_satellite_doxplex(ac_reqs)
converter, qubo = utils.convert_docplex_to_qubo(mdl)
print(qubo.prettyprint())
print(qubo.to_ising())